In [1]:
# Import các thư viện cần thiết
import numpy as np
import cv2
import pandas as pd
import os
import pickle
from google.colab import drive
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

In [3]:
# Connect Google Drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/TestOpCV'
# Hiển thị tất cả thư mục hoặc file
os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['models', 'face.png', 'FaceImage', 'data_face_features.pickle']

In [5]:
# Load data from pickle file
data = pickle.load(open("/content/drive/MyDrive/TestOpCV/data_face_features.pickle", mode="rb"))

In [6]:
x = np.array(data['data'])
y = np.array(data['label'])

print(x)
print(y)

[[[-0.00910952 -0.01439932 -0.09488683 ...  0.14309269 -0.10625915
    0.06958465]]

 [[-0.00338708  0.10263049  0.04596803 ... -0.00170245  0.11143715
    0.13530245]]

 [[-0.07862935  0.09107511  0.1608738  ...  0.06544042  0.18638234
    0.09604342]]

 ...

 [[ 0.06350037  0.19478916  0.00305566 ...  0.06762125  0.11532869
    0.03716978]]

 [[ 0.15373534  0.10608806 -0.05339221 ...  0.00700088 -0.03790517
   -0.00412309]]

 [[ 0.02926855  0.21446565 -0.03790818 ... -0.06640885  0.01369472
    0.06734287]]]
['1721031618_TruongThanhLuan' '1721031618_TruongThanhLuan'
 '1721031618_TruongThanhLuan' ... '151901205_BuiKhanhBinh'
 '151901205_BuiKhanhBinh' '151901205_BuiKhanhBinh']


In [7]:
x.shape

(4508, 1, 128)

In [8]:
y.shape

(4508,)

In [9]:
x = x.reshape(-1, 128)

In [10]:
x.shape

(4508, 128)

In [11]:
# chia nhỏ dữ liệu
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=1)

In [12]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((3606, 128), (902, 128), (3606,), (902,))

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [14]:
def get_report(model, x_train, y_train, x_test, y_test):
  y_pred_train = model.predict(x_train)
  y_pred_test = model.predict(x_test)

  # accuracy score
  acc_train = accuracy_score(y_train, y_pred_train)
  acc_test = accuracy_score(y_test, y_pred_test)

  # f1 score
  f1_score_train = f1_score(y_train, y_pred_train, average="macro")
  f1_score_test = f1_score(y_test, y_pred_test, average="macro")

  # print
  print('Accurency Train = %0.2f'%acc_train)
  print('Accurency Test = %0.2f'%acc_test)
  print('F1 Score Train = %0.2f'%f1_score_train)
  print('F1 Score Test = %0.2f'%f1_score_test)

In [15]:
# Logistic regression
model_logistic = LogisticRegression()
model_logistic.fit(x_train, y_train) # training

LogisticRegression()

In [16]:
get_report(model_logistic, x_train, y_train, x_test, y_test)

Accurency Train = 0.71
Accurency Test = 0.69
F1 Score Train = 0.50
F1 Score Test = 0.48


In [17]:
# Support Vector Machines
model_svc = SVC(probability=True)
model_svc.fit(x_train, y_train)

SVC(probability=True)

In [18]:
get_report(model_svc, x_train, y_train, x_test, y_test)

Accurency Train = 0.91
Accurency Test = 0.84
F1 Score Train = 0.83
F1 Score Test = 0.74


In [19]:
# Random forest
model_rf = RandomForestClassifier(n_estimators=20,)
model_rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=20)

In [20]:
get_report(model_rf, x_train, y_train, x_test, y_test)

Accurency Train = 1.00
Accurency Test = 0.76
F1 Score Train = 1.00
F1 Score Test = 0.65


In [21]:
# Voting Classifier
model_voting = VotingClassifier(estimators=[
    ('logistic', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('rf', RandomForestClassifier(n_estimators=20,)),
], voting='soft', weights=[2,3,1])

In [22]:
model_voting.fit(x_train, y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier(n_estimators=20))],
                 voting='soft', weights=[2, 3, 1])

In [23]:
get_report(model_voting, x_train, y_train, x_test, y_test)

Accurency Train = 0.98
Accurency Test = 0.84
F1 Score Train = 0.97
F1 Score Test = 0.78


In [24]:
# 3. Parameter Tuning
from sklearn.model_selection import GridSearchCV

model_grid = GridSearchCV(model_voting, param_grid={
    'svm__C': [3,5,7,10],
    'svm__gamma': [0.1, 0.3, 0.5],
    'rf__n_estimators': [5, 10,20],
    'rf__max_depth': [3, 5, 7],
    'voting': ['soft', 'hard']
}, scoring='accuracy', cv=3, n_jobs=1, verbose=2)

In [ ]:
model_grid.fit(x_train, y_train)

In [ ]:
model_best_estimator = model_grid.best_estimator_

In [ ]:
model_grid.best_score_

In [ ]:
# 4. Save Model
pickle.dump(model_best_estimator, open("/content/drive/MyDrive/TestOpCV/ml_face_person_identity.pkl", mode='wb'))